# Ejercicio 1 - Modelado de datos como RDF
## Temática elegida
La temática que se ha escogido para modelar los datos ha sido la serie de animación japonesa "*Serial Experiments: Lain*", un anime vanguardista conocido por tocar temas filosóficos y de comunicación social (centrado en el comienzo de internet en los 90-2000).

Se han modelado principalmente **personajes, organizaciones y objetos** que aparecen en a serie, intentando hacer incapié en las similitudes de esta con elementos de la vida real.

Entre los diferentes **conceptos y entidades** modelados, podemos encontrar:

- **Personajes**: los personajes principales de la serie, de tipo Person
- **Lugares**: (Solamente uno, ya que la trama transcurre en Tokyo, Japón)
- **Organizaciones**: tomando el termino de organización como grupo organizado, como pueden ser empresas o sectas
- **Productos/Objetos**: muy generalmente, todos aquellos elementos desarrollados por alguien con un fin o propósito, desde internet a productos de compra venta. Por enumerar algunos:
    - Código de programación
    - Dispositivos electrónicos
    - Ordenadores
    - Redes de ordenadores
- **Otros conceptos que aparecen representados**:
    - Dios: de manera simplificada, teniendo en cuenta su papel en la serie (mejor definido en la ontología)
    - Videojuegos: muy simplificadamente
    - Páginas web: tanto como enlaces como entidades en sí, ya que juegan un papel dentro de la serie
    - Conceptos físicos: como resonancias magnéticas
    - Eventos 

Para modelar este dominio, se ha hecho uso de **elementos de otras ontologías**, principalmente de Wikidata, Schema.org y DBpedia, aunque también se han tenido en cuenta otras ontologías como la [VideoGameOntology](http://purl.org/net/VideoGameOntology#), [FOAF](http://xmlns.com/foaf/0.1/) o [GEO](http://www.w3.org/2003/01/geo/wgs84_pos#). Entre estos elementos encontramos:
- La entidad de *schema:Person*, que representa a una persona
- La entidad *schema:Organization*, que representa organizaciones
- Se ha usado *schema:City* para representar ciudades
- Diversos Qs de Wikidata, entre ellos *Q15836568* (Computer Network) o *Q1293220* (Resonancia de Schumman)
- Para los eventos se ha empleado *schema:Event*
- Para el videojuego mencionado en el grafdo, se ha empleado *vgo:GameGenre* para describir su género

No se han empleado ontologías que describan a los personajes como personajes de una serie animada ni como peronajes de ficción ya que se ha considerado que no era pertinente para la aplicación que se le pretenderá dar a los datos del grafo.

In [ ]:
%pip install rdflib sparqlwrapper pyshex shexer --quiet --user
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from pyshex import ShExEvaluator
from pyshex.evaluate import evaluate

g = Graph()
base_path = "./lain.ttl"
g.parse(base_path)
g.serialize(format="turtle")

'@prefix dpedia: <http://dbpedia.org/resource/> .\n@prefix foaf: <http://xmlns.com/foaf/0.1/> .\n@prefix owl: <http://www.w3.org/2002/07/owl#> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix schema1: <http://schema.org/> .\n@prefix uo: <http://uniovi.es/miw/uo283069/entity/> .\n@prefix uop: <http://uniovi.es/miw/uo283069/property/> .\n@prefix vgo: <http://purl.org/net/VideoGameOntology#> .\n@prefix wd: <http://www.wikidata.org/entity/> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\nuo:Afx-n-nPrv-View.CL a uo:ComputerCode ;\n    schema1:creator uo:Lain_Iwakura ;\n    schema1:name "Afx-n-nPrv-View.CL" ;\n    schema1:programmingLanguage wd:Q849146 ;\n    uop:references wd:Q312 ;\n    rdfs:comment "Afx-n-nPrv-View.CL is the source for a code walker." ;\n    foaf:image <https://lain.wiki/images/a/ad/Walker.jpg> .\n\nuo:CRowView.CL a uo:ComputerCode ;\n    schema1:creator uo:Lain_Iwakura ;\n    schema

# Ejercicio 2 - Despliegue datos RDF + consultas SPARQL

- URL pública del endpoint de Apache Jena Fuseki: http://156.35.95.43:3030/SerialExperimentsLain/sparql
- URL pública del endpoint de Blazergaph: http://156.35.95.43:9999/blazegraph/namespace/kb/sparql (puede que no funcione por errores del servidor, fuseki en cambio si que funciona)


## Primera consulta: Familia
- Objetivo: Obtener los datos de personajes que formen una familia (padres, hijos o hermanos)
- Consulta NO FEDERADA
### Query:
```SPARQL
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?person ?personName (GROUP_CONCAT(DISTINCT ?relation; separator=", ") AS ?relations) (GROUP_CONCAT(DISTINCT ?relativeName; separator=", ") AS ?relatives)
WHERE {
  ?person schema:name ?personName .
  FILTER (LANG(?personName) = "en")

  {
    ?person schema:parent ?relative .
    ?relative schema:name ?relativeName .
    FILTER (LANG(?relativeName) = "en")
    BIND("Parent" AS ?relation)
  }
  UNION
  {
    ?person schema:sibling ?relative .
    ?relative schema:name ?relativeName .
    FILTER (LANG(?relativeName) = "en")
    BIND("Sibling" AS ?relation)
  }
  UNION
  {
    ?person schema:children ?relative .
    ?relative schema:name ?relativeName .
    FILTER (LANG(?relativeName) = "en")
    BIND("Child" AS ?relation)
  }
}
GROUP BY ?person ?personName
ORDER BY ?personName ?relation

```

In [7]:
str_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>

SELECT DISTINCT ?person ?personName (GROUP_CONCAT(DISTINCT ?relation; separator=", ") AS ?relations) (GROUP_CONCAT(DISTINCT ?relativeName; separator=", ") AS ?relatives)
WHERE {
  ?person schema:name ?personName .
  FILTER (LANG(?personName) = "en")

  {
    ?person schema:parent ?relative .
    ?relative schema:name ?relativeName .
    FILTER (LANG(?relativeName) = "en")
    BIND("Parent" AS ?relation)
  }
  UNION
  {
    ?person schema:sibling ?relative .
    ?relative schema:name ?relativeName .
    FILTER (LANG(?relativeName) = "en")
    BIND("Sibling" AS ?relation)
  }
  UNION
  {
    ?person schema:children ?relative .
    ?relative schema:name ?relativeName .
    FILTER (LANG(?relativeName) = "en")
    BIND("Child" AS ?relation)
  }
}
GROUP BY ?person ?personName
ORDER BY ?personName ?relation
"""
query_result = g.query(str_query)

# Los resultados se recorren por filas, quizá te recuerde a resultsets de librerías para ejecutar consultas SQL.
# Cada fila contiene objeto de dominio de rdflib (URIRef, Literal...) en variables que se llaman exactamente como los nombres de variables en la consulta SPARQL
for a_row in query_result:
  print(f"Person: {a_row.person}, Name: {a_row.personName}, Relations: {a_row.relations}, Relatives: {a_row.relatives}")

Person: http://example.org/Lain_Iwakura, Name: Lain Iwakura, Relations: Parent, Sibling, Relatives: Yasuo Iwakura, Miho Iwakura, Mika Iwakura
Person: http://example.org/MihoIwakura, Name: Miho Iwakura, Relations: Child, Relatives: Mika Iwakura, Lain Iwakura
Person: http://example.org/MikaIwakura, Name: Mika Iwakura, Relations: Parent, Sibling, Relatives: Miho Iwakura, Yasuo Iwakura, Lain Iwakura
Person: http://example.org/YasuoIwakura, Name: Yasuo Iwakura, Relations: Child, Relatives: Mika Iwakura, Lain Iwakura


## Segunda consulta: Referencias


```SPARQL
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX ex: <http://example.org/>

SELECT ?object ?name ?subject
WHERE {
  ?object ex:references ?subject .
  ?object schema:name ?name . 
}
```

In [4]:
str_query ="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX ex: <http://example.org/>

SELECT ?object ?name ?subject
WHERE {
  ?object ex:references ?subject .
  ?object schema:name ?name . 
}
"""
query_result = g.query(str_query)

for a_row in query_result:
  print(f"Object: {a_row.object}, Name: {a_row.name}, Subject: {a_row.subject}")

Object: http://example.org/Protocol6, Name: Protocol 6, Subject: http://www.wikidata.org/entity/Q2551624
Object: http://example.org/Protocol7, Name: Protocol 7, Subject: http://www.wikidata.org/entity/Q28823658
Object: http://example.org/KnightsoftheEasternCalculus, Name: Knights of the Eastern Calculus, Subject: http://www.wikidata.org/entity/Q6422517
Object: http://example.org/CoplandOS, Name: Copland OS, Subject: http://www.wikidata.org/entity/Q1131259
Object: http://example.org/NaviV1, Name: Child's NAVI, Subject: http://www.wikidata.org/entity/Q2141790
Object: http://example.org/Navi2, Name: NAVI, Subject: http://www.wikidata.org/entity/Q306381
Object: http://example.org/Navi3, Name: HandiNAVI, Subject: http://www.wikidata.org/entity/Q420772
Object: http://example.org/TheWired, Name: The Wired, Subject: http://www.wikidata.org/entity/Q466
Object: http://example.org/KensingtonExperiment, Name: Kensington Experiment, Subject: http://www.wikidata.org/entity/Q496036
Object: http://exa

## Tercera consulta: Federada
- Objetivo: Obtener, en base a la consulta anterior, los nombres de los sujetos en Wikidata

In [3]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://156.35.95.43:3030/SerialExperimentsLain/sparql")

str_query ="""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX uo: <http://uniovi.es/miw/uo283069/entity/> 
PREFIX uop: <http://uniovi.es/miw/uo283069/property/> 
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>

SELECT ?object ?name ?subject ?subjectLabel
WHERE {
    ?object uop:references ?subject .
    ?object schema:name ?name . 
    
    SERVICE <https://query.wikidata.org/sparql> {
        ?subject rdfs:label ?subjectLabel .
        FILTER (LANG(?subjectLabel) = "en")
    }
}
"""

sparql.setQuery(str_query)

sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    for a_var_key in result:
      print(result[a_var_key]["value"], end="       ")
    print()
print("--------")

http://uniovi.es/miw/uo283069/entity/Protocol6       Protocol 6       http://www.wikidata.org/entity/Q2551624       IPv6       
http://uniovi.es/miw/uo283069/entity/Protocol7       Protocol 7       http://www.wikidata.org/entity/Q28823658       TP/IX       
http://uniovi.es/miw/uo283069/entity/KnightsoftheEasternCalculus       Knights of the Eastern Calculus       http://www.wikidata.org/entity/Q6422517       Knights of the Lambda Calculus       
http://uniovi.es/miw/uo283069/entity/CoplandOS       Copland OS       http://www.wikidata.org/entity/Q1131259       Copland       
http://uniovi.es/miw/uo283069/entity/NaviV1       Child's NAVI       http://www.wikidata.org/entity/Q2141790       Twentieth Anniversary Macintosh       
http://uniovi.es/miw/uo283069/entity/Navi2       NAVI       http://www.wikidata.org/entity/Q306381       iMac G3       
http://uniovi.es/miw/uo283069/entity/Navi3       HandiNAVI       http://www.wikidata.org/entity/Q420772       Newton       
http://uniovi.es/miw

## Cuarta consula: Relaciones entre nodos

- Objetivo (un poco complejo): Comprobar si hay o no una relación entre dos entidades, para ello utilizaremos el oeprador del wildcard <>.
- No federada

In [67]:
"""
Explicación:
En cualquier tripleta, el predicado tiene que obligatoriamente ser <> o no serlo,
es decir (<>|!<>), si además a esto le añadimos el operador de relación inversa (^),
podemos decir que, en una relación entre dos entidades que estén conectadas, debe existir
una relación tal que (<>|!<>)|^(<>|!<>). Esto sumado al operador * que indica que 
la relación puede ser de cualquier longitud, nos permite saber si dos entidades están
conectadas o no. (En este caso, como el grafo es epqueño y todo está conectado, 
siempre va a haber una conexión entre dos entidades por muy larga que sea).
"""
ask_query = """
ASK {
    ex:MenInBlack ((<>|!<>)|^(<>|!<>))* ex:God
}
"""

ask_result = g.query(ask_query)

if ask_result.askAnswer:
    print("There is a connection between MenInBlack and God.")
else:
    print("There is no connection between MenInBlack and God.")

There is a connection between MenInBlack and God.


## Quinta consulta: Lugares de trabajo
- Objetivo: obtener los lugares de trabajo de cada persona que tenga uno, estos trabajos se representan con nodos anónimos .
- No federada

In [10]:
query = """
PREFIX schema: <http://schema.org/>

SELECT ?japaneseName ?personName ?occupationLabel ?placeOfOccupation
WHERE {
    ?person a schema:Person  .
    OPTIONAL { ?person schema:name ?japaneseName . FILTER (LANG(?japaneseName) = "ja") }
    ?person schema:name ?personName .
    ?person schema:hasOccupation ?occupation .
    ?occupation a schema:Occupation .
    ?occupation schema:name ?occupationLabel .
    OPTIONAL { ?person schema:employmentUnit ?placeOfOccupation }

    FILTER (LANG(?personName) = "en")
}
"""

query_result = g.query(query)

for a_row in query_result:
    print(f"Japanese Name: {a_row.japaneseName}, English Name: {a_row.personName}, Occupation: {a_row.occupationLabel}, Place of Occupation: {a_row.placeOfOccupation}\n")

Japanese Name: 岩倉 康男, English Name: Yasuo Iwakura, Occupation: Computer Programmer, Place of Occupation: http://example.org/TachibanaLabs

Japanese Name: 英利 政美, English Name: Masami Eiri, Occupation: Computer Programmer, Place of Occupation: http://example.org/TachibanaLabs

Japanese Name: None, English Name: Professor Hodgeson, Occupation: Scientist, Place of Occupation: None

Japanese Name: None, English Name: Karl Haushofer, Occupation: Spy, Place of Occupation: None

Japanese Name: None, English Name: Lin Sui-Xi, Occupation: Spy, Place of Occupation: None



## Sexta consulta - Información sobre un perosnaje
- Objetivo: construir una consulta federada que nos permita obtener información de Wikidata sobre un personaje del grafo.
- Ferderada

In [12]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://156.35.95.43:3030/SerialExperimentsLain/sparql")  # Construimos un objeto para acceder a cierto endpoint. En este caso, DBpedia
sparql.setQuery("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX uo: <http://uniovi.es/miw/uo283069/entity/>
PREFIX uop: <http://uniovi.es/miw/uo283069/property/> 
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT ?subject ?subjectLabel ?predicate ?object
WHERE {
  ?subject a schema:Person .
  ?subject owl:sameAs ?wikidataEntity .
  SERVICE <https://query.wikidata.org/sparql> {
    ?wikidataEntity rdfs:label ?subjectLabel .
    ?wikidataEntity ?predicate ?object .
    FILTER (LANG(?subjectLabel) = "en")
  }
}
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
    for a_var_key in result:
      print(result[a_var_key]["value"], end="       ")
    print()
print("--------")

http://uniovi.es/miw/uo283069/entity/Lain_Iwakura       Lain Iwakura       http://schema.org/version       2295876459       
http://uniovi.es/miw/uo283069/entity/Lain_Iwakura       Lain Iwakura       http://schema.org/dateModified       2025-01-07T14:41:36Z       
http://uniovi.es/miw/uo283069/entity/Lain_Iwakura       Lain Iwakura       http://schema.org/description       character in Serial Experiments Lain       
http://uniovi.es/miw/uo283069/entity/Lain_Iwakura       Lain Iwakura       http://schema.org/description       karakter utama pada Serial Experiments Lain       
http://uniovi.es/miw/uo283069/entity/Lain_Iwakura       Lain Iwakura       http://schema.org/description       personaggio immaginario e protagonista di Serial Experiments Lain       
http://uniovi.es/miw/uo283069/entity/Lain_Iwakura       Lain Iwakura       http://www.w3.org/2000/01/rdf-schema#label       Lain Iwakura       
http://uniovi.es/miw/uo283069/entity/Lain_Iwakura       Lain Iwakura       http://www.w3.o

## Séptima consulta - Número de objetos existentes
- Objetivo: obtener el número de elementos de tipo objeto que existen en el grafo
- No federada

In [14]:
query = """
PREFIX ex: <http://example.org/>
PREFIX schema: <http://schema.org/>
PREFIX dpedia: <http://dbpedia.org/resource/>

SELECT (COUNT(?object) AS ?count) 
WHERE {
  ?object a ?type .
  VALUES ?type { ex:ComputerCode schema:WebPage schema:VideoGame dpedia:Electrical_device dpedia:Computer_network dpedia:Computer dpedia:Operating_system }
}
"""

query_result = g.query(query)

for a_row in query_result:
    print(f"Number of objects: {a_row['count']}")

Number of objects: 9


## Ocatava consulta - Mayores de edad
- Objetivo: obtener las personas mayores de edad
- No federada

In [16]:
query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX schema: <http://schema.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?person ?name ?age
WHERE {
    ?person rdf:type schema:Person .
    ?person schema:name ?name .
    ?person foaf:age ?age .
    FILTER (?age > 18)
    FILTER(LANG(?name) = "en")
}
"""

query_result = g.query(query)

for a_row in query_result:
    print(f"Person: {a_row.person}, Name: {a_row.name}, Age: {a_row.age}")

Person: http://uniovi.es/miw/uo283069/entity/YasuoIwakura, Name: Yasuo Iwakura, Age: 45
Person: http://uniovi.es/miw/uo283069/entity/MihoIwakura, Name: Miho Iwakura, Age: 38
Person: http://uniovi.es/miw/uo283069/entity/MasamiEiri, Name: Masami Eiri, Age: 30
Person: http://uniovi.es/miw/uo283069/entity/ProfessorHodgeson, Name: Professor Hodgeson, Age: 60
Person: http://uniovi.es/miw/uo283069/entity/KarlHaushofer, Name: Karl Haushofer, Age: 32
Person: http://uniovi.es/miw/uo283069/entity/LinSuiXi, Name: Lin Sui-Xi, Age: 35


# Ejercicio 3: Desarrollo de aplicación web 

Url de la web: http://156.35.95.43:3000/

## Para desplegar la web:

En un servidor:
```powershell
docker build -t web-semantica-web-app .
docker run -d -p 3000:3000 web-semantica-web-app
```

En local:
```powershell
cmd /web-semantica
npm i
npm stat
```

## Descripción del contenido

Se ha tratado de ofrecer al usuario una forma "fácil" de navegar a través del grafo de RDF. La intención de la web es permitir al usuario visualizar todos los nodos **pertinentes** del grafo mostrandle la información de todas sus relaciones (exceptuando la de rdf:type).

El contenido de la web se basa en la serie japonesa escogida para modelar los datos, y sigue la siguiente lógica.

![Diagrama en resources/Web.png](/resources/Web.png)
                                          
## Descripción de la forma de uso de RDFa
Se ha empleado RDFa para describir los nodos representados en la pantalla del usuario de manera que cada vista representa el nodo de la siguiente forma si lo procesasemos con un procesador de RDFa:
```xml
@prefix rdfa: <http://www.w3.org/ns/rdfa#> .
@prefix schema: <http://schema.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

<https://rdfa.info/play/>
   rdfa:usesVocabulary schema: .
<http://uniovi.es/miw/uo283069/entity/Lain_Iwakura>
   rdf:type schema:Person;
   schema:name "Lain Iwakura, 岩倉 レイン"@en;
   foaf:gender "Female"@en;
   foaf:age "14"@en;
   schema:parent <http://uniovi.es/miw/uo283069/entity/YasuoIwakura>;
   schema:parent <http://uniovi.es/miw/uo283069/entity/MihoIwakura>;
   schema:sibling <http://uniovi.es/miw/uo283069/entity/MikaIwakura>;
   <http://uniovi.es/miw/uo283069/entity/characterRole> "Main Character"@en .
```
Esto nos permite a su misma vez tener tanto una representación visual del nodo como una visualiación de sus datos en RDFa.

# Ejercicio 4 - Validación datos RDF

Los ficheros utilizados se encuentran bajo la carpeta *shapes*, dentro de ella hay:
- CharacterShape: shape para personajes
- LocationShape: shpae para lugaress
- OccupationShape: shape para ocupación (trabajo)
- OrganizationShape: shape para organizaciones
- ComputerCodeShape: shape para programas informáticos
- SHapeMap.sm: EL shape map utilzado para validar los datos

In [13]:
with open("shapes/FullSHape.shex", "r") as file:
    shapes = file.read()

characters = ["http://uniovi.es/miw/uo283069/entity/Lain_Iwakura", "http://uniovi.es/miw/uo283069/entity/YasuoIwakura",
              "http://uniovi.es/miw/uo283069/entity/MihoIwakura", "http://uniovi.es/miw/uo283069/entity/MikaIwakura", 
              "http://uniovi.es/miw/uo283069/entity/MasamiEiri", "http://uniovi.es/miw/uo283069/entity/ProfessorHodgeson", 
              "http://uniovi.es/miw/uo283069/entity/KarlHaushofer", "http://uniovi.es/miw/uo283069/entity/LinSuiXi"]

for character in characters:
    evaluator = ShExEvaluator(rdf=g.serialize(format="turtle"), schema=shapes, 
                              focus=character, start="SchemaCharacter")
    results = evaluator.evaluate()

    for result in results:
        if not result.result:
            print(f"Node: {result.focus}, Conforms: {result.result}, Reason: {result.reason}")
        else: 
            print(f"Node: {result.focus}, Conforms: {result.result}")

Node: http://uniovi.es/miw/uo283069/entity/Lain_Iwakura, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/YasuoIwakura, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/MihoIwakura, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/MikaIwakura, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/MasamiEiri, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/ProfessorHodgeson, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/KarlHaushofer, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/LinSuiXi, Conforms: True


In [12]:
evaluator = ShExEvaluator(rdf=g.serialize(format="turtle"), schema=shapes, 
                          focus="http://uniovi.es/miw/uo283069/entity/Tokyo", 
                          start="SchemaLocation")
results = evaluator.evaluate()

for result in results:
    if not result.result:
        print(f"Node: {result.focus}, Conforms: {result.result}, Reason: {result.reason}")
    else: 
        print(f"Node: {result.focus}, Conforms: {result.result}")

Node: http://uniovi.es/miw/uo283069/entity/Tokyo, Conforms: True


In [14]:
nodes = ["http://uniovi.es/miw/uo283069/entity/TachibanaLabs", "http://uniovi.es/miw/uo283069/entity/MenInBlack",
         "http://uniovi.es/miw/uo283069/entity/KnightsoftheEasternCalculus"]

for node in nodes:
    evaluator = ShExEvaluator(rdf=g.serialize(format="turtle"), schema=shapes, 
                              focus=node, start="SchemaOrganization")
    results = evaluator.evaluate()

    for result in results:
        if not result.result:
            print(f"Node: {result.focus}, Conforms: {result.result}, Reason: {result.reason}")
        else: 
            print(f"Node: {result.focus}, Conforms: {result.result}")

Node: http://uniovi.es/miw/uo283069/entity/TachibanaLabs, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/MenInBlack, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/KnightsoftheEasternCalculus, Conforms: True


In [15]:
nodes = ["http://uniovi.es/miw/uo283069/entity/CRowView.CL", "http://uniovi.es/miw/uo283069/entity/Afx-n-nPrv-View.CL"]

for node in nodes:
    evaluator = ShExEvaluator(rdf=g.serialize(format="turtle"), schema=shapes, 
                              focus=node, 
                              start="ComputerCodeShape")
    results = evaluator.evaluate()

    for result in results:
        if not result.result:
            print(f"Node: {result.focus}, Conforms: {result.result}, Reason: {result.reason}")
        else: 
            print(f"Node: {result.focus}, Conforms: {result.result}")

Node: http://uniovi.es/miw/uo283069/entity/CRowView.CL, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/Afx-n-nPrv-View.CL, Conforms: True


In [16]:
evaluator = ShExEvaluator(rdf=g.serialize(format="turtle"), schema=shapes, 
                          focus="http://uniovi.es/miw/uo283069/entity/PHANTOMa", 
                          start="SchemaVideoGame")
results = evaluator.evaluate()

for result in results:
    if not result.result:
        print(f"Node: {result.focus}, Conforms: {result.result}, Reason: {result.reason}")
    else: 
        print(f"Node: {result.focus}, Conforms: {result.result}")

Node: http://uniovi.es/miw/uo283069/entity/PHANTOMa, Conforms: True


In [17]:
nodes = []
for a_triple in g.triples(  (None,
                             URIRef('http://schema.org/creator'),
                             None   )  ):
    s, p, o = a_triple
    nodes.append(s)

for node in nodes:
    evaluator = ShExEvaluator(rdf=g.serialize(format="turtle"), schema=shapes, 
                              focus=node, 
                              start="CreatedItemShape")
    results = evaluator.evaluate()

    for result in results:
        if not result.result:
            print(f"Node: {result.focus}, Conforms: {result.result}, Reason: {result.reason}")
        else: 
            print(f"Node: {result.focus}, Conforms: {result.result}")

Node: http://uniovi.es/miw/uo283069/entity/Protocol7, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/KIDS, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/Mebious, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/CRowView.CL, Conforms: True
Node: http://uniovi.es/miw/uo283069/entity/Afx-n-nPrv-View.CL, Conforms: True


### Shape map
Se puede encontrar en **/SHEX/ShapeMap.sm** su contenido se presenta a continuación:
```RDF
SPARQL"""
PREFIX schema: <http://schema.org/>
select ?node where{
	?node a schema:Person .
	FILTER(isIRI(?node))
}
"""@<SchemaCharacter>,
uo:Tokyo@<SchemaLocation>,
SPARQL """
PREFIX schema: <http://schema.org/>
PREFIX uop: <http://uniovi.es/miw/uo283069/property/> 
select ?node where {
?node a schema:Organization
}
"""@<SchemaOrganization>,
{FOCUS a uo:ComputerCode}@<ComputerCodeShape>,
SPARQL"""
PREFIX schema: <http://schema.org/>
select ?item where {
?item schema:creator ?person
}
"""@<CreatedItemShape>,
uo:PHANTOMa@<SchemaVideoGame>
```
### SHACL
En **/SHEX/ShapesSHACL.ttl** se pueden encontrar las mismas shapes escritas en SHACL.

## Ejercicio 5 - Extracción de shapes

In [ ]:
from shexer.shaper import Shaper
from shexer.consts import TURTLE


namespaces_dict = {
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#": "rdf",
    "http://example.org/": "ex",
    "http://www.w3.org/2001/XMLSchema#": "xsd",
    "http://schema.org/": "schema",
    "http://xmlns.com/foaf/0.1/": "foaf",
    "http://purl.org/dc/terms/": "dcterms",
    "http://uniovi.es/miw/uo283069/entity/": "uo",
    "http://uniovi.es/miw/uo283069/property/": "uop",
    "http://www.wikidata.org/entity/": "wd",
    "http://dbpedia.org/resource/": "dpedia",
    "http://www.w3.org/2003/01/geo/wgs84_pos#": "geo",
    "https://www.onetonline.org/link/summary/": "onet",
    "http://www.w3.org/2000/01/rdf-schema#": "rdfs",
    "http://www.w3.org/2002/07/owl#": "owl",
    "http://purl.org/net/VideoGameOntology#": "vgo"
}

shaper = Shaper(graph_file_input="./utf8_lain.ttl",
                input_format=TURTLE,  
                all_classes_mode=True,
                disable_comments=False,
                namespaces_dict=namespaces_dict)

result = shaper.shex_graph(string_output=True, acceptance_threshold=0.2)  
print(result)

Las shapes en SHEX se pueden encontrar en **/SHEX/ShexerShapes.shex**.
Si comparamos las Shapes extraidas, podemos ver algunas diferncias con las nuestras:

En verde y rojo las diferencias entre mi Shape (verde) y la obtenida con shexer (rojo) (Puede que en GitHub no aparezcan los colores, en todo caso, en rojo son aquellos con un '-' y en verde aquellos con un '+')
```diff
:Person {
-    schema:name  xsd:string  *;
+    schema:name rdf:langString + ;      
-    foaf:age  xsd:integer  ?;    
+    foaf:age xsd:integer ;  
-    uop:characterRole  xsd:string  ?;   
+    uop:characterRole xsd:string + ;  
-    rdf:type  [schema:Person]  ?; 
+    rdf:type [schema:Person uo:God] {1,2} ;
-    foaf:gender  xsd:string  ?  
+    foaf:gender xsd:string + ; 
     schema:parent  @:Person  *;
-    schema:sibling  @:Person  ? ;
+    schema:sibling @<SchemaCharacter> * ;
     schema:children  @:Person  *;
+    schema:follows @<SchemaCharacter> * ;
-    schema:hasOccupation  BNode  ?;
+    schema:hasOccupation @<SchemaOccupation> ? ;
-    schema:employmentUnit  IRI  ?;
+    schema:employmentUnit @<SchemaOrganization> ? ;
-    schema:deathPlace  IRI  ?;
+    schema:deathPlace @<SchemaLocation> ?  ;     
}
```
Como podemos ver, para esta shape, que es la más comlpeta de todas en el grafo, el extractor de shapes se deja una sin deducir `schema:follows` y difiere en cuanto shape en los tipos de datos de algunas tripletas como en `schema:name` y en la cardinalidad de las mismas en otras, aunque consigue acertar `schema:parent` y `schema:children`.

Si miramos otras shapes más cortas, como la de Organización

```diff
:Organization
{
   schema:name  xsd:string  ;   
   rdf:type  [schema:Organization]  ;
   schema:location  @:City  ?;
-   uop:references  IRI  ?;
-   schema:founder  BNode  ?;
+   schema:founder {
+    a [schema:Person] ; 
+    schema:name rdf:langString * ; 
+    schema:employmentUnit @<SchemaOrganization> ? ; 
+    foaf:gender xsd:string ? ;
+   } ? ;
}
```
Podemos ver que no difiere tanto de la shape creada por mí, en este caso añade `uop:references`, lo cual no estaría mal pero no forma una parte intrínseca de la descrpición de organización, sino que es más una meta-realción que busca describir una referencia del propio nodo a un objeto o entidad real, por lo que no consta en mi esquema. Además es incapaz de describir el `schema:founder` como un nodo anónimo que se componga de diferentes relaciones, por lo que es posible que a la hora de extraer shapes no podamos realizar inferencias de mayor nivel sobre nodos que desconocemos en el grafo, necesitando aquí el apoyo humano.

El resto de shapes son muy simples, por lo que no merece excesivamente la pena compararlas, sin embargo, es notable remarcar una cosa, si no se lo especificamos explicitamente, el extractor es incapaz de inferir que cualquier clase que sea una creación o un objeto manufacturado, debe cumplir con una única shape como *CreatedItemsShape*, en este caso podemos especificarselo nosotros manualmente, pero ya tenemos que decirle específicamente que queremos todas las clases que cumplen con la condición de ser creadas por alguien. 

En mi opinión personal, creo que estas herramientas tienen mucho potencial en aquellos casos en los que se quiera extraer una shape de un concepto del que se tengan unas bases claras, y que resultan sobre todo útiles para ayudarnos a darnos cuenta de elementos que tal vez nosotros no hayamos tenido en cuenta a la hora de crear nuestra Shape pero que pueden ser de interés a la hora de describir dichos conceptos.

Como pensamiento espontaneo, otra manera en la que se puede aplicar esto puede ser a modo de investigación sobre las relaciones de algunos constructos, si podemos decir (como ejemplo muy simplificado) que de todas las entidades de Wikidata que son de tipo "Construcción" y que hayan sido construidas por una persona X presentan con un 80% de frecuencia una propiedad "Derumbado" o "Destruido", tal vez se puedan realizar conjeturas sobre la causalidad de este mismo.

# Ejercicio 6 -  Elaboración de ontología
La ontología se puede encontrar en el archivo **ontology/ontology.owl** enb la raíz del proyecto (el archivo ontology/ontology.ttl es el generado por Protegé para crear la ontología).

En cuanto a las dierencias apreciables entre la idea de ontología y de shapes, es bastante obvio resaltar la funcionalidad que tiene cada uno. Una ontología es un "vocabulario" de términos que pretenede especificar *qué* términos se deben utilizar y junto con *qué* propiedades o entidades, mientras que las shapes espeficican *cómo* se debe componer un elemento en un grafo para representar un concepto determinado.

El contexto en el que se debe aplicar una ontología sería entonces en el momento de describir los datos, sobretodo si buscamos interoperabilidad con el resto de datos pre-existentes y representar conceptos que estén fuertemente relacionados con los de otros grafos de conocimiento. Es en el momento en el que decidimos *qué* datos vamos a representar en el que debemos decidir *en qué* manera representarlos, y si otras ontologías describen nuestros conceptos de una manera que se adaptea nuestras necesidades. En cambio, las shapes se aplicarían al mometno de validar datos para comprobar que estos están estructurados y siguen un patrón definido por nosotros.

No obstante, considero que existe un punto medio donde ambos conceptos "cooperan" y compensan sus dos objetivos. Imaginemos que tenemos una aplicaciónque guarda datos médicos en RDF, y que queremos que esos datos sean interoperables y estén relacionados con conceptos médicos definidos en ontologías externas, sería lógico pensar que entonces tenemos que hacer uso de esas ontologías, pero ¿cómo permitimos al usuario (considerando que no es experto en RDF, y por tanto no va a insertar datos con sentencias SPARQL) introducir nueva información en el grafo de información sin romper las Shapes que tengamos definidas? Aquí sería donde entran en juego los dos conceptos, necesitaremos una herramienta que permita al usuario introducir datos que cumplan con nuestra shape y que además haga uso de las ontologías (probablemente especificadas en la propia shape). Si esto se hiciese, habría que prestar especial atención en las shapes abiertas para permitir al usuario introducir más información sobre un concepto sin dejar de utilizar las ontologías especializadas, aunque esto ya es divagar en profundiad a cerca del tema.

Conclusivamente, aunque existen diferentes contextos en los que se debe emplear una ontología o una shape, estos dos conceptos no son mutuamente exclusivos, sino que se complementan entre si para expandir los grafos de conocimiento.

# Ejercicio 7 - Grafos de conocimiento y LLMs

Tomamos como primer ejemplo la segunda consulta del ejercicio 2 sobre referencias de nuestro grafo a elementos externos. Al estár representados con URIs externas, es muy simple obtener información a cerca de estas con una consulta federada. 

Sorprendentemente, un LLM como ChatGPT es capaz de obtener la misma información del grafo que nosotros realizando una query con SPARQL **una vez provisto del grafo de conocimiento**, esto claro, ya que hace uso de la librería *rdflib* para obtener los mismos datos que nosotros, por lo que realmente lo único que está haciendo es "copiar" nuestro código sin consultar la información en fuentes externas.

Link a la conversación: https://chatgpt.com/share/67b0c15f-0f24-800b-b1bd-bab105fa48f3

Si probamos en cambio a no ofrecerle una base de información, podemos ver que obtiene mucha menos información de la que nosotros le hemos provisto, y que en algunos caso se le escapan algunos conceptos (Knights of the Eastern Calculus == Knights of the Lambda Calculus) o que nos dá información muy rebuscada que se escapa del prompt (Eiri Masami → Transhumanist Thinkers). Además de esto no nos proporciona las fuentes que ha empleado para obtener esta información.

Link a la conversación: https://chatgpt.com/share/67b0c29a-10c8-800b-a3d7-be6790fb2b9a

La primera conversación con ChatGPT nos da una pequeña pista a cerca de un tema comentado en el paper: ["Large Language Models, Knowledge Graphs and Search Engines: A Crossroad for Answering Users' QUestions"](https://arxiv.org/abs/2501.06699), en el que se habla del uso conjunto entre LLMs y Grafos de conocimiento.

En ese artículo, se mencionan tres maneras de combinar estas tecnologías para reforzar su manera de obtener conocimiento:

- **LLMs for Knowledge Graphs**: consistiría en "rellenar" los "huecos" de los grafos de información con la información obtenida con un LLM, más adelante probaremos a hacer una demostración práctica de esto.
- **Knowledge Graphs for LLMs**: consiste en proporcionar a los LLMs con grafos de conocimiento para que puedan *validar* la información que le van a dar al usuario, esto podría asemejarse a lo que hemos hecho en el primer chat con ChatGPT, ya que toda la información que nos ha dado provenía de ese mismo grafo.
- **KG + LLM + SE**: consiste en combinar las tres teconologías para obtener información mediante un proceso "largo", similar a un *Map-Reduce*. Como resulta un poco más complejo, no entraremos en detalle en este caso, pero un ejemplo de su aplicación podría verse en mi presentación para la asignatura en el pdf [*WS_LLMs_KG.pdf*](./WS_LLMs_KG.pdf) en la página 10, en la que ChatGPT emplea un Search Engine para acceder a información de un Grafo de Conocimiento.

Como ejemplo de LLMs for KG, se ha tenido una conversación con ChatGPT para pedirle que rellen el grafo con información faltante. Se le ha pedido explícitamente que se centre solo en los personajes, y que devuelva la información en RDF con sintaxis de Turtle para poder compararla con nuestro grafo actual. El resultado a sido el siguiente archivo [/llms/rdf.ttl](./llms/rdf.ttl)

Como podemos apreciar, aunque sí que **nos ofrece información que faltaba** en el grafo (Por ejemplo el personaje Taro), **nos da información incorrecta** (Karl Haushofer no es el "Top executive" de los hombres de negro y este segundo no es una Persona, si no una organización) y **no emplea ni los prefijos ni las relaciones existentes** del grafo que le habíamos dado como base (Por ejemplo, no pone que el padre y la madre de Lain_Iwakura tengan una relación con ella de tipo *children*).

Además de esto, se le ha pedido que genere una shape con SHEX para validar los nodos de personajes, ya que esta también podría ser otra aplicación de los LLMs en el campo de los grafos de conocimiento. Aunque el schema generado no es del todo incorrecto, si lo comparamos con el que especificamos nosotros en [shapes/CharacterShape](./shapes/CharacterShape.shex), podemos ver que hay ciertas relaciones que le faltan, además de añadir las que anteriormente utilizó él cuando nos dió más información en vez de centrarse solo en el grafo que le pasamos.


```shex
PREFIX schema: <http://schema.org/>
PREFIX ex: <http://example.org/lain/>

ex:CharacterShape {
    schema:name xsd:string+ ;   # Every character must have at least one name
    schema:age xsd:string? ;   # Age is optional
    schema:affiliation IRI* ;  # A character may belong to multiple organizations
    schema:role xsd:string* ;  # Role(s) of the character (optional, multiple allowed)
    schema:description xsd:string* ;  # Character description(s)
    schema:parent IRI* ;  # Links to parent(s)
    schema:sibling IRI* ;  # Links to sibling(s)
    schema:voiceActor xsd:string* ;  # Multiple voice actors possible
}

```

Link a la conversación: https://chatgpt.com/share/67b0cdef-0420-800b-b782-df7433e626d3

Como conlcusión aquí, podemos decir que aún falta mucho recorrido para que estas tecnologías se entiendan y complementen entre sí, pero no parece una idea muy descabellada pensar que en un futuro podrían llegar a emplearse de alguna de las maneras descritas para potenciar el concimiento en la web.

# Ejercicio 8
La presentación realizad en clase se encuentra en [*WS_LLMs_KG.pdf*](./WS_LLMs_KG.pdf).

La bilbiografía de la misma se puede resumir en el siguiente artículo: ["Large Language Models, Knowledge Graphs and Search Engines: A Crossroad for Answering Users' QUestions"](https://arxiv.org/abs/2501.06699).